# NHL Player Projections

Using machine learning and previous seasons' statistics to predict a player's statistics for the upcoming year.

Note: Data is from Natural Stat Trick.

Note 2: Ignoring goalies (for now).

## Notes for me

1. Going to have to worry about NaNs and -. Can probably just fill everything with 0's but need to consider if a player does not play in a particular season. - Using left join now, might help with this.
2. We have one extra player in the 2018-2019 bio than in 2018-2019 stats. Rookie? Retired player? Injury?

## Table of Contents

1. [Importing Python Libraries](#Importing-Python-Libraries)
2. [Reading the Data](#Reading-the-Data)


## Importing Python Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

## Reading the Data

In [3]:
path = '../data/'

In [4]:
# Function that will create a total dataframe for each type of situation and for a specified season
# Index condition is re
def read_situation_data(sit, season, index=True):
    
    '''
    Helper function that will create a total dataframe for each type of situation and for a specified season.
    Note: Index variable is really just there to make merging all the situations together easier.
    
    '''

    # Read individual data
    df_individual = pd.read_csv(path + season + ' ' + sit + ' ' + 'Individual.csv')
    df_individual.drop('Unnamed: 0', axis=1, inplace=True)

    # Read on ice data
    df_on_ice = pd.read_csv(path + season + ' ' + sit + ' ' + 'On-Ice.csv')
    df_on_ice.drop('Unnamed: 0', axis=1, inplace=True)

    # List of overlapping columns that we will merge on
    overlapping_cols = ['Player', 'Team', 'Position', 'GP', 'TOI']

    # Merge
    df_sit = pd.merge(df_individual, df_on_ice, how='outer', left_on=overlapping_cols, right_on=overlapping_cols)

    # Need to map a code to the column names so that we can easily them the situation from it.
    # However, not going to prefix 'Player', 'Team', 'Position', 'GP' as these do not vary with the situations
    codes_dict = {'All Strengths':'AS', 'Even Strength':'ES', 'Power Play':'PP', 'Penalty Kill':'PK'}
    df_sit.columns = [codes_dict[sit] + ' ' + str(col) for col in df_sit.columns]
    df_sit.rename(columns={codes_dict[sit] + ' Player':'Player',
                           codes_dict[sit] + ' Team':'Team',
                           codes_dict[sit] + ' Position':'Position',
                           codes_dict[sit] + ' GP':'GP'},inplace=True)
    
    # Set index to 'Player' based on condition
    if index:
        df_sit.set_index('Player', inplace=True)

    return df_sit

In [5]:
# read_situation_data('All Strengths', '2018-2019')

In [6]:
def read_season_data(season, index=True):
    
    '''
    Helper function that will create a total dataframe for a specified season.
    Stats are for the following situations: Even Strength (ES), Power Play (PP), and Penalty Kill (PK)
    For now, we will not aggregate (a.k.a. add All Situations (AS))
    Note: Similarly, index variable is really just there to make merging all the situations together easier.
    
    '''
    
    # List of overlapping columns that we will merge on
    overlapping_cols = ['Player', 'Team', 'Position', 'GP']
    
    # Use helper function to create dataframes for all 3 situations and merge on the overlapping columns.
    df_season = read_situation_data('Even Strength', season, index=False)
    df_season = pd.merge(df_season, read_situation_data('Power Play', season, index=False), how='outer', 
                         left_on=overlapping_cols, right_on=overlapping_cols)
    df_season = pd.merge(df_season, read_situation_data('Penalty Kill', season, index=False), how='outer', 
                         left_on=overlapping_cols, right_on=overlapping_cols)
    
    # Add season prefix to each column. For readability, we will only use the later year.
    # (i.e. for the 2015-2016 season, each column will start with 2016)
    later_year = season.split('-')[1]
    df_season.columns = [later_year + ' ' + str(col) for col in df_season.columns]
    df_season.rename(columns={later_year + ' Player':'Player'},inplace=True)
    
    # Set index to 'Player' based on condition
    if index:
        df_season.set_index('Player', inplace=True)
    
    return df_season

In [7]:
# read_season_data('2018-2019')

In [8]:
def read_data(seasons_list=['2015-2016','2016-2017','2017-2018','2018-2019']):
    
    '''
    This is the function that will read the data required for building the model.
    We will use the previous 3 years of the data to predict stats for the upcoming season.
    Also, unlike the previous read_data helper functions, we will add bio information but only for the most recent season in the list.
    This means, the dataset will have each player's biography for the season in which we will predict stats.
    Note: Codes for different situations: Even Strength (ES), Power Play (PP), and Penalty Kill (PK)
    Note 2: Because I'm only interested in players playing in the upcoming year, going to use a left join.
    
    '''
    
    # Just in case, sort seasons_list
    seasons_list.sort()
    
    # Read bio
    max_year = max(seasons_list)
    df = pd.read_csv(path + max_year + ' ' + 'Player Bios.csv', index_col='Player')
    df.drop('Unnamed: 0', axis=1, inplace=True)
    
    # Iterate through the seasons and merge their corresponding dataframes to df
    for season in seasons_list:
        df_season = read_season_data(season, index=True)
        df = pd.merge(df, df_season, how='left', left_index=True, right_index=True)
        
    return df

In [9]:
df = read_data()
display(df.head())
display(df.shape)

,Team,Position,Age,Date of Birth,Birth City,Birth State/Province,Birth Country,Nationality,Height (in),Weight (lbs),Draft Year,Draft Team,Draft Round,Round Pick,Overall Draft Position,2016 Team,2016 Position,2016 GP,2016 ES TOI,2016 ES Goals,2016 ES Total Assists,2016 ES First Assists,2016 ES Second Assists,2016 ES Total Points,2016 ES IPP,2016 ES Shots,2016 ES SH%,2016 ES ixG,2016 ES iCF,2016 ES iFF,2016 ES iSCF,2016 ES iHDCF,2016 ES Rush Attempts,2016 ES Rebounds Created,2016 ES PIM,2016 ES Total Penalties,2016 ES Minor,2016 ES Major,2016 ES Misconduct,2016 ES Penalties Drawn,2016 ES Giveaways,2016 ES Takeaways,2016 ES Hits,2016 ES Hits Taken,2016 ES Shots Blocked,2016 ES Faceoffs Won,2016 ES Faceoffs Lost,2016 ES Faceoffs %,2016 ES CF,2016 ES CA,2016 ES CF%,2016 ES FF,2016 ES FA,2016 ES FF%,2016 ES SF,2016 ES SA,2016 ES SF%,2016 ES GF,2016 ES GA,2016 ES GF%,2016 ES xGF,2016 ES xGA,2016 ES xGF%,2016 ES SCF,2016 ES SCA,2016 ES SCF%,2016 ES HDCF,2016 ES HDCA,2016 ES HDCF%,2016 ES HDGF,2016 ES HDGA,2016 ES HDGF%,2016 ES MDCF,2016 ES MDCA,2016 ES MDCF%,2016 ES MDGF,2016 ES MDGA,2016 ES MDGF%,2016 ES LDCF,2016 ES LDCA,2016 ES LDCF%,2016 ES LDGF,2016 ES LDGA,2016 ES LDGF%,2016 ES On-Ice SH%,2016 ES On-Ice SV%,2016 ES PDO,2016 ES Off. Zone Starts,2016 ES Neu. Zone Starts,2016 ES Def. Zone Starts,2016 ES On The Fly Starts,2016 ES Off. Zone Start %,2016 ES Off. Zone Faceoffs,2016 ES Neu. Zone Faceoffs,2016 ES Def. Zone Faceoffs,2016 ES Off. Zone Faceoff %,2016 PP TOI,2016 PP Goals,2016 PP Total Assists,2016 PP First Assists,...,2019 PP MDCF%,2019 PP MDGF,2019 PP MDGA,2019 PP MDGF%,2019 PP LDCF,2019 PP LDCA,2019 PP LDCF%,2019 PP LDGF,2019 PP LDGA,2019 PP LDGF%,2019 PP On-Ice SH%,2019 PP On-Ice SV%,2019 PP PDO,2019 PP Off. Zone Starts,2019 PP Neu. Zone Starts,2019 PP Def. Zone Starts,2019 PP On The Fly Starts,2019 PP Off. Zone Start %,2019 PP Off. Zone Faceoffs,2019 PP Neu. Zone Faceoffs,2019 PP Def. Zone Faceoffs,2019 PP Off. Zone Faceoff %,2019 PK TOI,2019 PK Goals,2019 PK Total Assists,2019 PK First Assists,2019 PK Second Assists,2019 PK Total Points,2019 PK IPP,2019 PK Shots,2019 PK SH%,2019 PK ixG,2019 PK iCF,2019 PK iFF,2019 PK iSCF,2019 PK iHDCF,2019 PK Rush Attempts,2019 PK Rebounds Created,2019 PK PIM,2019 PK Total Penalties,2019 PK Minor,2019 PK Major,2019 PK Misconduct,2019 PK Penalties Drawn,2019 PK Giveaways,2019 PK Takeaways,2019 PK Hits,2019 PK Hits Taken,2019 PK Shots Blocked,2019 PK Faceoffs Won,2019 PK Faceoffs Lost,2019 PK Faceoffs %,2019 PK CF,2019 PK CA,2019 PK CF%,2019 PK FF,2019 PK FA,2019 PK FF%,2019 PK SF,2019 PK SA,2019 PK SF%,2019 PK GF,2019 PK GA,2019 PK GF%,2019 PK xGF,2019 PK xGA,2019 PK xGF%,2019 PK SCF,2019 PK SCA,2019 PK SCF%,2019 PK HDCF,2019 PK HDCA,2019 PK HDCF%,2019 PK HDGF,2019 PK HDGA,2019 PK HDGF%,2019 PK MDCF,2019 PK MDCA,2019 PK MDCF%,2019 PK MDGF,2019 PK MDGA,2019 PK MDGF%,2019 PK LDCF,2019 PK LDCA,2019 PK LDCF%,2019 PK LDGF,2019 PK LDGA,2019 PK LDGF%,2019 PK On-Ice SH%,2019 PK On-Ice SV%,2019 PK PDO,2019 PK Off. Zone Starts,2019 PK Neu. Zone Starts,2019 PK Def. Zone Starts,2019 PK On The Fly Starts,2019 PK Off. Zone Start %,2019 PK Off. Zone Faceoffs,2019 PK Neu. Zone Faceoffs,2019 PK Def. Zone Faceoffs,2019 PK Off. Zone Faceoff %
Player,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A.J. Greer,COL,L,22,1996-12-14,Joliette,QC,CAN,CAN,75,210,2015,COL,2,9,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

(907, 963)